## Model training and tuning for the energy sector

In [1]:
import pandas as pd
import numpy as np

import time

from darts import TimeSeries, concatenate
from darts.utils.model_selection import train_test_split
from darts.models.forecasting.xgboost import XGBModel
from darts.dataprocessing.transformers.scaler import Scaler
from darts.metrics.metrics import mape, mae
from darts.models.forecasting.prophet_model import Prophet
from darts.models.forecasting.sf_auto_theta import AutoTheta
from darts.models.forecasting.linear_regression_model import LinearRegressionModel
from darts.dataprocessing.transformers import StaticCovariatesTransformer
from darts.models.forecasting.sf_auto_arima import AutoARIMA
from darts.dataprocessing.transformers.boxcox import BoxCox

Importing plotly failed. Interactive plots will not work.
d:\investing-bot\.venv\Lib\site-packages\fs\__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore


In [2]:
df = pd.read_csv('../../data/processed/sp500_all_companies_cleaned_with_sector.csv')
df['date'] = pd.to_datetime(df['date'])

df_energy = df[df['GICS Sector'] == 'Energy'].sort_values(by=['ticker', 'date']).reset_index(drop=True)

In [3]:
df['GICS Sector'].value_counts()

GICS Sector
Industrials               193459
Financials                190485
Information Technology    177149
Health Care               151348
Consumer Discretionary    130185
Consumer Staples           96649
Real Estate                79757
Utilities                  78786
Materials                  63394
Energy                     58523
Communication Services     58017
Name: count, dtype: int64

In [4]:
df_energy['ticker'].unique()

array(['APA', 'BKR', 'COP', 'CTRA', 'CVX', 'DVN', 'EOG', 'EQT', 'EXE',
       'FANG', 'HAL', 'HES', 'KMI', 'MPC', 'OKE', 'OXY', 'PSX', 'SLB',
       'TPL', 'TRGP', 'VLO', 'WMB', 'XOM'], dtype=object)

In [5]:
data = {}
static_covs = pd.DataFrame({'ticker': df_energy['ticker'].unique()})

for ticker, group in df_energy.groupby('ticker'):
    series = TimeSeries.from_dataframe(
        group,
        time_col='date',
        value_cols=['close', 'high', 'low', 'open', 'volume'],
        freq='B',
        static_covariates=pd.Series(ticker)
    )
    data[ticker] = series

In [6]:
# encode each ticker as a static covariate
transformer = StaticCovariatesTransformer()
ts = [d for d in data.values()]
transformer.fit(ts)

for ticker, series in data.items():
    data[ticker] = transformer.transform(series)

In [7]:
train = {}
test = {}

for ticker, series in data.items():
    train[ticker], test[ticker] = train_test_split(series, test_size=0.1)
    last_train = train[ticker][-1:]
    test_with_context = last_train.append(test[ticker]) # append last point of train to test as our forecast requires it
    test[ticker] = test_with_context

## Gradient Boosting

### Train a local model first

In [8]:
models = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = XGBModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models[ticker] = model

Training model for APA
Model for APA trained in 0.35 seconds
**************************************************
Training model for BKR
Model for BKR trained in 0.36 seconds
**************************************************
Training model for COP
Model for COP trained in 0.35 seconds
**************************************************
Training model for CTRA
Model for CTRA trained in 0.42 seconds
**************************************************
Training model for CVX
Model for CVX trained in 0.36 seconds
**************************************************
Training model for DVN
Model for DVN trained in 0.40 seconds
**************************************************
Training model for EOG
Model for EOG trained in 0.34 seconds
**************************************************
Training model for EQT
Model for EQT trained in 0.34 seconds
**************************************************
Training model for EXE
Model for EXE trained in 0.30 seconds
*****************************************

In [9]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes[ticker] = mape_
    maes[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: APA, MAPE: 15.14, MAE: 3.44
**************************************************
Ticker: BKR, MAPE: 9.30, MAE: 3.90
**************************************************
Ticker: COP, MAPE: 3.54, MAE: 3.60
**************************************************
Ticker: CTRA, MAPE: 2.84, MAE: 0.72
**************************************************
Ticker: CVX, MAPE: 2.58, MAE: 3.86
**************************************************
Ticker: DVN, MAPE: 8.52, MAE: 2.99
**************************************************
Ticker: EOG, MAPE: 3.69, MAE: 4.57
**************************************************
Ticker: EQT, MAPE: 11.03, MAE: 5.19
**************************************************
Ticker: EXE, MAPE: 15.80, MAE: 16.42
**************************************************
Ticker: FANG, MAPE: 13.25, MAE: 20.78
**************************************************
Ticker: HAL, MAPE: 9.48, MAE: 2.63
**************************************************
Ticker: HES, MAPE: 3.30, MAE: 4.58
***********

In [10]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(10.91087322333438, 22.92285567658511)

### Global model

In [11]:
print(f"Training global model")
start = time.time()
model = XGBModel(
    lags=15,
    lags_past_covariates=1,
    lags_future_covariates=[0],
    output_chunk_length=1,
    random_state=42,
    add_encoders={
        'datetime_attribute': {
            'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
        }
    },
    use_static_covariates=True,
    n_estimators=1000,
    eta=0.01,
    max_depth=8,
    subsample=0.8
)

model.fit(
    [ts['close'] for ts in train.values()],
    past_covariates=[ts[['high', 'low', 'volume', 'open']] for ts in train.values()],
)

end = time.time()
print(f"Global model trained in {end - start:.2f} seconds")
print("*" * 50)

Training global model
Global model trained in 10.15 seconds
**************************************************


In [12]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_pred = model.predict(
        n=len(test[ticker]['close']),
        series=train[ticker]['close'],
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    y_test = test[ticker]['close']
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)

    mapes[ticker] = mape_
    maes[ticker] = mae_

    print(f"Global model for {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Global model for APA, MAPE: 6.70, MAE: 1.40
**************************************************
Global model for BKR, MAPE: 4.26, MAE: 1.67
**************************************************
Global model for COP, MAPE: 2.86, MAE: 2.90
**************************************************
Global model for CTRA, MAPE: 3.79, MAE: 0.98
**************************************************
Global model for CVX, MAPE: 1.91, MAE: 2.85
**************************************************
Global model for DVN, MAPE: 5.19, MAE: 1.84
**************************************************
Global model for EOG, MAPE: 2.52, MAE: 3.09
**************************************************
Global model for EQT, MAPE: 4.44, MAE: 1.81
**************************************************
Global model for EXE, MAPE: 2.62, MAE: 2.69
**************************************************
Global model for FANG, MAPE: 3.17, MAE: 5.37
**************************************************
Global model for HAL, MAPE: 9.53, MAE: 2.60
****

In [13]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(5.22117487188746, 22.670671761947055)

## Prophet model

In [14]:
models_prophet = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = Prophet(
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year']  # automatically generate future covariates from time series
            }
        },
        seasonality_mode='multiplicative',
    )

    model.fit(
        train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_prophet[ticker] = model

Training model for APA


22:11:37 - cmdstanpy - INFO - Chain [1] start processing
22:11:40 - cmdstanpy - INFO - Chain [1] done processing


Model for APA trained in 3.41 seconds
**************************************************
Training model for BKR


22:11:40 - cmdstanpy - INFO - Chain [1] start processing
22:11:43 - cmdstanpy - INFO - Chain [1] done processing


Model for BKR trained in 3.34 seconds
**************************************************
Training model for COP


...

Model for VLO trained in 2.47 seconds
**************************************************
Training model for WMB


22:12:31 - cmdstanpy - INFO - Chain [1] start processing
22:12:33 - cmdstanpy - INFO - Chain [1] done processing


Model for WMB trained in 2.51 seconds
**************************************************
Training model for XOM


22:12:34 - cmdstanpy - INFO - Chain [1] start processing
22:12:36 - cmdstanpy - INFO - Chain [1] done processing


Model for XOM trained in 3.09 seconds
**************************************************


In [15]:
mapes_prophet = {}
maes_prophet = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_prophet[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_prophet[ticker] = mape_
    maes_prophet[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: APA, MAPE: 14.07, MAE: 3.22
**************************************************
Ticker: BKR, MAPE: 14.07, MAE: 5.84
**************************************************
Ticker: COP, MAPE: 17.22, MAE: 17.25
**************************************************
Ticker: CTRA, MAPE: 8.99, MAE: 2.29
**************************************************
Ticker: CVX, MAPE: 17.48, MAE: 26.55
**************************************************
Ticker: DVN, MAPE: 10.55, MAE: 4.35
**************************************************
Ticker: EOG, MAPE: 5.41, MAE: 6.63
**************************************************
Ticker: EQT, MAPE: 21.60, MAE: 9.46
**************************************************
Ticker: EXE, MAPE: 4.50, MAE: 4.57
**************************************************
Ticker: FANG, MAPE: 16.01, MAE: 26.07
**************************************************
Ticker: HAL, MAPE: 27.60, MAE: 7.33
**************************************************
Ticker: HES, MAPE: 9.66, MAE: 14.02
*****

In [16]:
np.mean(list(mapes_prophet.values())), np.mean(list(maes_prophet.values()))

(19.740006349173587, 42.625915129539024)

## Theta model

In [17]:
models_theta = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = AutoTheta(
        season_length=30
    )

    model.fit(
        series=train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_theta[ticker] = model

Training model for APA
Model for APA trained in 0.67 seconds
**************************************************
Training model for BKR
Model for BKR trained in 0.57 seconds
**************************************************
Training model for COP
Model for COP trained in 0.64 seconds
**************************************************
Training model for CTRA
Model for CTRA trained in 0.56 seconds
**************************************************
Training model for CVX
Model for CVX trained in 0.72 seconds
**************************************************
Training model for DVN
Model for DVN trained in 0.59 seconds
**************************************************
Training model for EOG
Model for EOG trained in 0.62 seconds
**************************************************
Training model for EQT
Model for EQT trained in 0.59 seconds
**************************************************
Training model for EXE
Model for EXE trained in 0.13 seconds
*****************************************

In [18]:
mapes_theta = {}
maes_theta = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_theta[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_theta[ticker] = mape_
    maes_theta[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: APA, MAPE: 22.20, MAE: 4.48
**************************************************
Ticker: BKR, MAPE: 16.94, MAE: 6.99
**************************************************
Ticker: COP, MAPE: 17.23, MAE: 17.18
**************************************************
Ticker: CTRA, MAPE: 6.94, MAE: 1.71
**************************************************
Ticker: CVX, MAPE: 6.80, MAE: 9.82
**************************************************
Ticker: DVN, MAPE: 29.26, MAE: 10.39
**************************************************
Ticker: EOG, MAPE: 6.27, MAE: 7.43
**************************************************
Ticker: EQT, MAPE: 14.72, MAE: 6.49
**************************************************
Ticker: EXE, MAPE: 3.70, MAE: 3.87
**************************************************
Ticker: FANG, MAPE: 14.01, MAE: 21.67
**************************************************
Ticker: HAL, MAPE: 26.24, MAE: 6.89
**************************************************
Ticker: HES, MAPE: 13.56, MAE: 18.59
*****

In [19]:
np.mean(list(mapes_theta.values())), np.mean(list(maes_theta.values()))

(17.520389607814526, 34.180697366954945)

## Linear Regression

### Local model

In [20]:
models_lr = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = LinearRegressionModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_lr[ticker] = model

Training model for APA
Model for APA trained in 0.13 seconds
**************************************************
Training model for BKR
Model for BKR trained in 0.12 seconds
**************************************************
Training model for COP
Model for COP trained in 0.12 seconds
**************************************************
Training model for CTRA
Model for CTRA trained in 0.12 seconds
**************************************************
Training model for CVX
Model for CVX trained in 0.12 seconds
**************************************************
Training model for DVN
Model for DVN trained in 0.12 seconds
**************************************************
Training model for EOG
Model for EOG trained in 0.12 seconds
**************************************************
Training model for EQT
Model for EQT trained in 0.12 seconds
**************************************************
Training model for EXE
Model for EXE trained in 0.08 seconds
*****************************************

In [21]:
mapes_lr = {}
maes_lr = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_lr[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_lr[ticker] = mape_
    maes_lr[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: APA, MAPE: 4239.77, MAE: 752.50
**************************************************
Ticker: BKR, MAPE: 6.91, MAE: 2.80
**************************************************
Ticker: COP, MAPE: 3.07, MAE: 3.08
**************************************************
Ticker: CTRA, MAPE: 3.17, MAE: 0.82
**************************************************
Ticker: CVX, MAPE: 2.49, MAE: 3.68
**************************************************
Ticker: DVN, MAPE: 497.30, MAE: 164.19
**************************************************
Ticker: EOG, MAPE: 4.00, MAE: 4.77
**************************************************
Ticker: EQT, MAPE: 6.04, MAE: 2.58
**************************************************
Ticker: EXE, MAPE: 3.62, MAE: 3.78
**************************************************
Ticker: FANG, MAPE: 5251.24, MAE: 7464.78
**************************************************
Ticker: HAL, MAPE: 46603428021.60, MAE: 9843473835.75
**************************************************
Ticker: HES, MAPE:

In [22]:
np.mean(list(mapes_lr.values())), np.mean(list(maes_lr.values()))

(2041438292.2030005, 448422526.6429812)

### Global model

In [23]:
print(f"Training global model")
start = time.time()
model = LinearRegressionModel(
    lags=15,
    lags_past_covariates=1,
    lags_future_covariates=[0],
    output_chunk_length=1,
    random_state=42,
    add_encoders={
        'datetime_attribute': {
            'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
        }
    }
)

model.fit(
    [ts['close'] for ts in train.values()],
    past_covariates=[ts[['high', 'low', 'volume', 'open']] for ts in train.values()],
)

end = time.time()
print(f"Global model trained in {end - start:.2f} seconds")
print("*" * 50)

Training global model
Global model trained in 2.82 seconds
**************************************************


In [24]:
mapes_lr_global = {}
maes_lr_global = {}

for ticker in data.keys():
    y_pred = model.predict(
        n=len(test[ticker]['close']),
        series=train[ticker]['close'],
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    y_test = test[ticker]['close']
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)

    mapes_lr_global[ticker] = mape_
    maes_lr_global[ticker] = mae_

    print(f"Global model for {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Global model for APA, MAPE: 4.50, MAE: 0.97
**************************************************
Global model for BKR, MAPE: 2.86, MAE: 1.10
**************************************************
Global model for COP, MAPE: 2.69, MAE: 2.72
**************************************************
Global model for CTRA, MAPE: 3.03, MAE: 0.77
**************************************************
Global model for CVX, MAPE: 2.34, MAE: 3.46
**************************************************
Global model for DVN, MAPE: 3.54, MAE: 1.28
**************************************************
Global model for EOG, MAPE: 2.51, MAE: 3.06
**************************************************
Global model for EQT, MAPE: 3.08, MAE: 1.30
**************************************************
Global model for EXE, MAPE: 2.37, MAE: 2.43
**************************************************
Global model for FANG, MAPE: 3.16, MAE: 5.20
**************************************************
Global model for HAL, MAPE: 3.79, MAE: 1.03
****

In [25]:
np.mean(list(mapes_lr_global.values())), np.mean(list(maes_lr_global.values()))

(2.940330709259289, 4.5979221883562635)

## ARIMA

In [26]:
models_arima = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = AutoARIMA()

    model.fit(
        series=train[ticker]['close'],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_arima[ticker] = model

Training model for APA
Model for APA trained in 0.22 seconds
**************************************************
Training model for BKR
Model for BKR trained in 0.17 seconds
**************************************************
Training model for COP
Model for COP trained in 0.88 seconds
**************************************************
Training model for CTRA
Model for CTRA trained in 0.19 seconds
**************************************************
Training model for CVX
Model for CVX trained in 1.39 seconds
**************************************************
Training model for DVN
Model for DVN trained in 1.82 seconds
**************************************************
Training model for EOG
Model for EOG trained in 0.20 seconds
**************************************************
Training model for EQT
Model for EQT trained in 0.22 seconds
**************************************************
Training model for EXE
Model for EXE trained in 0.16 seconds
*****************************************

In [27]:
mapes_arima = {}
maes_arima = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_arima[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_arima[ticker] = mape_
    maes_arima[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: APA, MAPE: 22.56, MAE: 4.55
**************************************************
Ticker: BKR, MAPE: 17.35, MAE: 7.15
**************************************************
Ticker: COP, MAPE: 18.24, MAE: 18.18
**************************************************
Ticker: CTRA, MAPE: 6.36, MAE: 1.59
**************************************************
Ticker: CVX, MAPE: 4.71, MAE: 6.84
**************************************************
Ticker: DVN, MAPE: 27.44, MAE: 9.72
**************************************************
Ticker: EOG, MAPE: 5.08, MAE: 6.03
**************************************************
Ticker: EQT, MAPE: 14.73, MAE: 6.54
**************************************************
Ticker: EXE, MAPE: 4.71, MAE: 4.94
**************************************************
Ticker: FANG, MAPE: 12.77, MAE: 19.74
**************************************************
Ticker: HAL, MAPE: 26.48, MAE: 6.94
**************************************************
Ticker: HES, MAPE: 11.01, MAE: 15.05
******

In [28]:
np.mean(list(mapes_arima.values())), np.mean(list(maes_arima.values()))

(16.951430424435262, 34.148155106253554)